# Camada Bronze - Extração dos dados

In [0]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
%python
# Check if the file exists
if dbutils.fs.ls('/FileStore/tables/TelecomX_Data.json'):
    # Read the JSON file
    url = '/dbfs/FileStore/tables/TelecomX_Data.json'
    df_raw = pd.read_json(url)
    display(df_raw.head())
else:
    print("File does not exist")

# Transformação - Camada Silver

In [0]:
# Verifica se as colunas aninhadas existem antes de expandir
colunas_aninhadas = ['customer', 'phone', 'internet', 'account']
colunas_existentes = [col for col in colunas_aninhadas if col in df_raw.columns]
# Expande apenas as colunas que existem
df_expandido = pd.concat(
    [df_raw.drop(columns=colunas_existentes)] +
    [df_raw[col].apply(pd.Series) for col in colunas_existentes if col in df_raw.columns],
    axis=1
)


In [0]:
# Renomeia se a coluna existir
if "customerID" in df_expandido.columns:
    df_expandido.rename(columns={"customerID": "id_cliente"}, inplace=True)
    # Expandindo coluna Charges, se existir
if 'Charges' in df_expandido.columns:
    charges_expandido = df_expandido['Charges'].apply(pd.Series)
    df_expandido = pd.concat([
        df_expandido.drop(columns=['Charges']),
        charges_expandido
    ], axis=1)

# Renomear colunas após expandir
df_expandido.rename(columns={
    'Monthly': 'valor_mensal',
    'Total': 'valor_total',
    'gender': 'Genero',
    'SeniorCitizen': 'Idoso',
    'Partner': 'Parceiro',
    'Dependents': 'Dependentes',
    'tenure': 'Tempo de serviço',
    'PhoneService': 'Telefone',
    'MultipleLines': 'Multiplas linhas',
    'InternetService': 'Servico de internet',
    'OnlineSecurity': 'Seguranca online',
    'OnlineBackup': 'Backup',
    'DeviceProtection': 'Protecao de dispositivo',
    'TechSupport': 'Suporte tecnico',
    'StreamingTV': 'Streaming TV',
    'StreamingMovies': 'Streaming Filmes',
    'Contract': 'Contrato',
    'PaperlessBilling': 'Faturamento',
    'PaymentMethod': 'Metodo de pagamento'
}, inplace=True)

# Conversão numérica com tratamento de valores ausentes
if 'valor_mensal' in df_expandido.columns:
    df_expandido['valor_mensal'] = pd.to_numeric(df_expandido['valor_mensal'], errors='coerce').fillna(0)
if 'valor_total' in df_expandido.columns:
    df_expandido['valor_total'] = pd.to_numeric(df_expandido['valor_total'], errors='coerce').fillna(0)

print(df_expandido.head())

In [0]:
# Substitui valores em branco por NaN
df_expandido.replace('', pd.NA, inplace=True)

# Remove caracteres especiais de todas as colunas de string
for col in df_expandido.select_dtypes(include='object').columns:
    df_expandido[col] = df_expandido[col].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)) if pd.notna(x) else x)

# Remove duplicatas
df_expandido.drop_duplicates(inplace=True)

print(df_expandido.head())

In [0]:
# Criar a coluna 'Contas_Diarias' dividindo o 'valor_mensal' por 30 (aproximadamente o número de dias em um mês)
df_expandido['Contas_Diarias'] = df_expandido['valor_mensal'] / 30

print(df_expandido.head())

In [0]:
# Função para transformar 'Sim'/'Não' em 1/0
def transformar_sim_nao(df, colunas):
  for coluna in colunas:
    if coluna in df.columns:
      df[coluna] = df[coluna].apply(lambda x: 1 if x == 'Yes' else (0 if x == 'No' else x))
  return df

# Transformar colunas 'Sim'/'Não' em 1/0
colunas_sim_nao = ['Parceiro', 'Dependentes', 'Telefone', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup',
                   'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
df_expandido = transformar_sim_nao(df_expandido, colunas_sim_nao)


# Transformar a coluna 'Genero' em 'Masculino' e 'Feminino'
if 'Genero' in df_expandido.columns:
  df_expandido['Genero'] = df_expandido['Genero'].apply(lambda x: 'Feminino' if x == 'Female' else ('Masculino' if x == 'Male' else x))

# Transformar a coluna 'Idoso' em 'Sim' e 'Não'
if 'Idoso' in df_expandido.columns:
  df_expandido['Idoso'] = df_expandido['Idoso'].apply(lambda x: 'Sim' if x == 1 else ('Não' if x == 0 else x))

print(df_expandido.head())

# Anáises exploratória e visualização - Camada Gold

In [0]:
# Realizando a análise descritiva para as colunas numéricas
# Selecionando apenas as colunas numéricas para a análise descritiva
df_numeric = df_expandido.select_dtypes(include=['number'])

# Calculando as métricas descritivas
analise_descritiva = df_numeric.describe()

# Exibindo a análise descritiva
print("\nAnálise Descritiva das Colunas Numéricas:")
display(analise_descritiva)

In [0]:
# Para as colunas categóricas, podemos verificar a contagem de valores únicos
df_categorical = df_expandido.select_dtypes(include=['object'])

print("\nAnálise Descritiva das Colunas Categóricas (Contagem de Valores):")
for col in df_categorical.columns:
    print(f"\nColuna: {col}")
    display(df_categorical[col].value_counts())

In [0]:
def plot_Churn_distribution(df):
    if 'Churn' not in df.columns:
        print("Coluna 'Churn' não encontrada.")
        return
    df_filtered = df[df['Churn'].isin(['Yes', 'No'])]
    plt.figure(figsize=(6,4))
    sns.countplot(data=df_filtered, x='Churn', palette={'Yes':'blue', 'No':'red'})
    plt.title('Distribuição de Churn - Telecom X')
    plt.xlabel('Churn')
    plt.ylabel('Contagem')
    plt.show()

plot_Churn_distribution(df_expandido)

In [0]:
import matplotlib.pyplot as plt
def plot_categorical_churn_distribution(df, categorical_col):
    if 'Churn' not in df.columns:
        print("Coluna 'Churn' não encontrada.")
        return
    if categorical_col not in df.columns:
        print(f"Coluna '{categorical_col}' não encontrada.")
        return

    df_filtered = df[df['Churn'].isin(['Yes', 'No'])].copy()
    # Remove linhas onde a coluna categórica é NaN
    df_filtered.dropna(subset=[categorical_col], inplace=True)


    plt.figure(figsize=(10, 6))
    sns.countplot(data=df_filtered, x=categorical_col, hue='Churn', palette={'Yes':'blue', 'No':'gray'})
    plt.title(f'Distribuição de Evasão por {categorical_col}')
    plt.xlabel(categorical_col)
    plt.ylabel('Contagem')
    plt.xticks(rotation=45, ha='right') # Rotaciona os rótulos do eixo X para melhor visualização
    plt.tight_layout()
    plt.show()

# Lista de colunas categóricas para analisar
categorical_columns_to_analyze = [
    'Genero',
    'Idoso',
    'Parceiro',
    'Dependentes',
    'Telefone',
    'Multiplas linhas',
    'Servico de internet',
    'Seguranca online',
    'Backup',
    'Protecao de dispositivo',
    'Suporte tecnico',
    'Streaming TV',
    'Streaming Filmes',
    'Contrato',
    'Faturamento',
    'Metodo de pagamento'
]

# Itera sobre as colunas categóricas e plota a distribuição de evasão
for col in categorical_columns_to_analyze:
    plot_categorical_churn_distribution(df_expandido, col)